# Homework 3: Prediction and Classification

Due: Thursday, October 16, 2014 11:59 PM

<a href=https://raw.githubusercontent.com/cs109/2014/master/homework/HW3.ipynb download=HW3.ipynb> Download this assignment</a>

#### Submission Instructions
To submit your homework, create a folder named lastname_firstinitial_hw# and place your IPython notebooks, data files, and any other files in this folder. Your IPython Notebooks should be completely executed with the results visible in the notebook. We should not have to run any code. Compress the folder (please use .zip compression) and submit to the CS109 dropbox in the appropriate folder. If we cannot access your work because these directions are not followed correctly, we will not grade your work.

---


# Introduction

In this assignment you will be using regression and classification to explore different data sets.  

**First**: You will use data from before 2002 in the [Sean Lahman's Baseball Database](http://seanlahman.com/baseball-archive/statistics) to create a metric for picking baseball players using linear regression. This is same database we used in Homework 1. This database contains the "complete batting and pitching statistics from 1871 to 2013, plus fielding statistics, standings, team stats, managerial records, post-season data, and more". [Documentation provided here](http://seanlahman.com/files/database/readme2012.txt).

!["Sabermetrics Science"](http://saberseminar.com/wp-content/uploads/2012/01/saber-web.jpg)
http://saberseminar.com/wp-content/uploads/2012/01/saber-web.jpg

**Second**: You will use the famous [iris](http://en.wikipedia.org/wiki/Iris_flower_data_set) data set to perform a $k$-neareast neighbor classification using cross validation.  While it was introduced in 1936, it is still [one of the most popular](http://archive.ics.uci.edu/ml/) example data sets in the machine learning community. Wikipedia describes the data set as follows: "The data set consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimetres." Here is an illustration what the four features measure:

!["iris data features"](http://sebastianraschka.com/Images_old/2014_python_lda/iris_petal_sepal.png)
http://sebastianraschka.com/Images_old/2014_python_lda/iris_petal_sepal.png
http://sebastianraschka.com/Images/2014_python_lda/iris_petal_sepal.png

**Third**: You will investigate the influence of higher dimensional spaces on the classification using another standard data set in machine learning called the The [digits data set](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html).  This data set is similar to the MNIST data set discussed in the lecture. The main difference is, that each digit is represented by an 8x8 pixel image patch, which is considerably smaller than the 28x28 pixels from MNIST. In addition, the gray values are restricted to 16 different values (4 bit), instead of 256 (8 bit) for MNIST. 

**Finally**: In preparation for Homework 4, we want you to read through the following articles related to predicting the 2014 Senate Midterm Elections. 

* [Nate Silver's Methodology at while at NYT](http://fivethirtyeight.blogs.nytimes.com/methodology/)
* [How The FiveThirtyEight Senate Forecast Model Works](http://fivethirtyeight.com/features/how-the-fivethirtyeight-senate-forecast-model-works/)
* [Pollster Ratings v4.0: Methodology](http://fivethirtyeight.com/features/pollster-ratings-v40-methodology/)
* [Pollster Ratings v4.0: Results](http://fivethirtyeight.com/features/pollster-ratings-v40-results/)
* [Nate Silver versus Sam Wang](http://www.washingtonpost.com/blogs/plum-line/wp/2014/09/17/nate-silver-versus-sam-wang/)
* [More Nate Silver versus Sam Wang](http://www.dailykos.com/story/2014/09/09/1328288/-Get-Ready-To-Rumbllllle-Battle-Of-The-Nerds-Nate-Silver-VS-Sam-Wang)
* [Nate Silver explains critisims of Sam Wang](http://politicalwire.com/archives/2014/10/02/nate_silver_rebuts_sam_wang.html)
* [Background on the feud between Nate Silver and Sam Wang](http://talkingpointsmemo.com/dc/nate-silver-sam-wang-feud)
* [Are there swing voters?]( http://www.stat.columbia.edu/~gelman/research/unpublished/swing_voters.pdf)



---

## Load Python modules

In [ ]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

# If this module is not already installed, you may need to install it. 
# You can do this by typing 'pip install seaborn' in the command line
import seaborn as sns 

import sklearn
import sklearn.datasets
import sklearn.cross_validation
import sklearn.decomposition
import sklearn.grid_search
import sklearn.neighbors
import sklearn.metrics

# Problem 1: Sabermetrics

Using data preceding the 2002 season pick 10 offensive players keeping the payroll under $20 million (assign each player the median salary). Predict how many games this team would win in a 162 game season.  

In this problem we will be returning to the [Sean Lahman's Baseball Database](http://seanlahman.com/baseball-archive/statistics) that we used in Homework 1.  From this database, we will be extract five data sets containing information such as yearly stats and standing, batting statistics, fielding statistics, player names, player salaries and biographical information. You will explore the data in this database from before 2002 and create a metric for picking players. 

#### Problem 1(a) 

Load in [these CSV files](http://seanlahman.com/files/database/lahman-csv_2014-02-14.zip) from the [Sean Lahman's Baseball Database](http://seanlahman.com/baseball-archive/statistics). For this assignment, we will use the 'Teams.csv', 'Batting.csv', 'Salaries.csv', 'Fielding.csv', 'Master.csv' tables. Read these tables into separate pandas DataFrames with the following names. 

CSV file name | Name of pandas DataFrame
:---: | :---: 
Teams.csv | teams
Batting.csv | players
Salaries.csv | salaries
Fielding.csv | fielding
Master.csv | master

In [ ]:
### Your code here ###
import requests, zipfile, StringIO
r = requests.get("http://seanlahman.com/files/database/lahman-csv_2014-02-14.zip")
z = zipfile.ZipFile(StringIO.StringIO(r.content), 'r')
#z.namelist()

In [ ]:
teams = pd.read_csv(z.open('Teams.csv'))        # shape = (2745, 48)
players = pd.read_csv(z.open('Batting.csv'))    # shape = (97889, 24)
salaries = pd.read_csv(z.open('Salaries.csv'))  # shape = (23956, 5)
fielding = pd.read_csv(z.open('Fielding.csv'))  # shape = (166991, 18)
master = pd.read_csv(z.open('Master.csv'))      # shape = (18354, 24)

#### Problem 1(b)

Calculate the median salary for each player and create a pandas DataFrame called `medianSalaries` with four columns: (1) the player ID, (2) the first name of the player, (3) the last name of the player and (4) the median salary of the player. Show the head of the `medianSalaries` DataFrame.   

In [ ]:
medianSalaries0 = salaries.groupby(['playerID'], as_index=False)['playerID','salary'].median()
medianSalaries0.head()

In [ ]:
medianSalaries = pd.merge(master[['playerID', 'nameGiven', 'nameLast']], 
                          medianSalaries0, 
                          how='inner', on='playerID')
medianSalaries.head()

#### Problem 1(c)

Now, consider only team/season combinations in which the teams played 162 Games. Exclude all data from before 1947. Compute the per plate appearance rates for singles, doubles, triples, HR, and BB. Create a new pandas DataFrame called `stats` that has the teamID, yearID, wins and these rates.

**Hint**: Singles are hits that are not doubles, triples, nor HR. Plate appearances are base on balls plus at bats.

In [ ]:
# filter pre-1947 and != 162 games
filteredteams = teams[(teams.yearID > 1947) & (teams.G == 162)] # shape = (984, 48)
#filteredteams.info()

In [ ]:
# Compute the per plate appearance rates for singles, doubles, triples, HR, and BB. 
# Create a new pandas DataFrame called stats that has the teamID, yearID, wins and these rates.
# Hint: Singles are hits that are not doubles, triples, nor HR. Plate appearances are base on balls plus at bats.

filteredteams = filteredteams[['teamID','yearID', 'W', 'H', '2B', '3B', 'HR', 'AB', 'BB']]

# (H - (2B + 3B + HR)), 2B, 3B, HR, BB
# PA = BB + AB

filteredteams['PA'] = filteredteams.AB + filteredteams.BB
filteredteams['Singles'] = filteredteams.H - (filteredteams['2B'] + filteredteams['3B'] + filteredteams.HR)

for col in ['Singles', '2B', '3B', 'HR', 'BB']:
    filteredteams[col] = filteredteams[col] / filteredteams['PA']

filteredteams.rename(columns={'Singles':'SinglesPA', 
                              '2B':'DoublesPA', 
                              '3B':'TriplesPA', 
                              'HR':'HRPA', 
                              'BB':'BBPA'}, inplace = True)
stats = filteredteams[['teamID','yearID', 'W', 'SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']]

stats.head()

In [ ]:
stats.hist('yearID', bins=stats['yearID'].unique())

#### Problem 1(d)

Is there a noticeable time trend in the rates computed in Problem 1(c)? 

In [ ]:
for col in ['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']:
    stats.plot(x='yearID', y=col, kind='scatter', alpha=0.5, title=col)

#### Problem 1(e) 

Using the `stats` DataFrame from Problem 1(c), adjust the singles per PA rates so that the average across teams for each year is 0. Do the same for the doubles, triples, HR, and BB rates. 

In [ ]:
print(stats[['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']].mean().apply(lambda x: round(x, 3)))
#stats[['Singles/PA_adj','2B/PA_adj','3B/PA_adj','HR/PA_adj']] = stats.groupby('yearID')[['Singles/PA','2B/PA','3B/PA','HR/PA']].apply(lambda x: x-x.mean())
stats[['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']] = stats.groupby('yearID')[['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']].apply(lambda x: x-x.mean())
print(stats[['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']].mean().apply(lambda x: round(x, 3)))

#def meanNormalizeRates(df):
#        subRates = df[["1B","2B","3B","HR","BB"]]
#        df[["1B","2B","3B","HR","BB"]] = subRates - subRates.mean(axis=0)
#        return df
#stats = stats.groupby('yearID').apply(meanNormalizeRates)

In [ ]:
for col in ['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']:
    stats.plot(x='yearID', y=col, kind='scatter', c='orange', alpha=0.5, title=col)

In [ ]:
for col in ['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']:
    stats[stats['yearID']==2002].plot(x='W', y=col, kind='scatter', c='g', title=col)

#### Problem 1(f)

Build a simple linear regression model to predict the number of wins from the average adjusted singles, double, triples, HR, and BB rates. To decide which of these terms to include fit the model to data from 2002 and compute the average squared residuals from predictions to years past 2002. Use the fitted model to define a new sabermetric summary: offensive predicted wins (OPW). Hint: the new summary should be a linear combination of one to five of the five rates.


In [ ]:
sns.regplot(y="W", x="SinglesPA", data=stats2002, fit_reg = True, color = 'blue')
#sns.regplot(y="W", x="DoublesPA", data=stats2002, fit_reg = True, color = 'green')
#sns.regplot(y="W", x="TriplesPA", data=stats2002, fit_reg = True, color = 'orange')
sns.regplot(y="W", x="HRPA", data=stats2002, fit_reg = True, color = 'grey')
sns.regplot(y="W", x="BBPA", data=stats2002, fit_reg = True, color = 'red')

In [ ]:
sns.regplot(y="W", x="DoublesPA", data=stats2002, fit_reg = True, color = 'green')
sns.regplot(y="W", x="TriplesPA", data=stats2002, fit_reg = True, color = 'orange')

In [ ]:
#stats[stats['yearID'] == 2002].drop(['teamID', 'yearID', 'W'], axis='columns')

In [ ]:
from sklearn.linear_model import LinearRegression
trainX = stats[stats['yearID'] < 2002].drop(['teamID', 'yearID', 'W'], axis='columns')
trainY = stats[stats['yearID'] < 2002]['W']
testX = stats[stats['yearID'] >= 2002].drop(['teamID', 'yearID', 'W'], axis='columns')
testY = stats[stats['yearID'] >= 2002]['W']
clf = LinearRegression()
clf.fit(trainX, trainY)

In [ ]:
print 'Estimated intercept coefficient:', clf.intercept_
print 'Number of coefficients:', len(clf.coef_)
# The coefficients
pd.DataFrame(zip(trainX.columns, clf.coef_), columns = ['features', 'estimatedCoefficients'])

In [ ]:
print 'Estimated intercept coefficient:', clf.intercept_
print 'Number of coefficients:', len(clf.coef_)
# The coefficients
pd.DataFrame(zip(trainX.columns, clf.coef_), columns = ['features', 'estimatedCoefficients'])

In [ ]:
trainY.hist(color="grey")
plt.hist(clf.predict(trainX))
plt.title('Predicted Wins (fitted values, blue): $\hat{Y}_i$')
plt.xlabel('Wins')
plt.ylabel('Teams')

In [ ]:
plt.scatter(trainY, clf.predict(trainX))
plt.xlabel("Wins: $Y_i$")
plt.ylabel("Predicted wins: $\hat{Y}_i$")
plt.title("Wins vs Predicted Wins: $Y_i$ vs $\hat{Y}_i$")
plt.plot( [70,95], [70,95], c='gray', ls='--' ) # reference line, manual 
plt.show()

In [ ]:
print np.sum((testY - clf.predict(testX)) ** 2)

In [ ]:
print("Mean squared error: %.2f"
      % np.mean((testY - clf.predict(testX)) ** 2))

In [ ]:
plt.scatter(testY, clf.predict(testX), c='orange')
plt.xlabel("Wins: $Y_i$")
plt.ylabel("Predicted wins: $\hat{Y}_i$")
plt.title("Wins vs Predicted Wins: $Y_i$ vs $\hat{Y}_i$")
plt.plot( [50,100], [50,100], c='gray', ls='--' ) # reference line 45*, manual 
plt.show()

In [ ]:
plt.scatter(testY, clf.predict(testX), c='orange')
plt.xlabel("Wins: $Y_i$")
plt.ylabel("Predicted wins: $\hat{Y}_i$")
plt.title("Wins vs Predicted Wins: $Y_i$ vs $\hat{Y}_i$")
plt.plot( [50,100], [50,100], c='gray', ls='--' ) # reference line 45*, manual 
plt.show()

In [ ]:
plotX = stats[stats['yearID'] >= 2002].copy()
plotX['OPW'] = clf.predict(plotX.drop(['teamID', 'yearID', 'W'], axis='columns'))

sns.regplot(y="OPW", x="W", data=plotX, fit_reg = True, color = 'orange')
sns.regplot(x="W", y="W", data=plotX, fit_reg= True, color = 'gray', scatter=False)

In [ ]:
plotX = stats[stats['yearID'] >= 2002].copy()
plotX['OPW'] = clf.predict(plotX.drop(['teamID', 'yearID', 'W'], axis='columns'))

sns.regplot(y="OPW", x="W", data=plotX, fit_reg = True, color = 'orange')
sns.regplot(x="W", y="W", data=plotX, fit_reg= True, color = 'gray', scatter=False)

##### Alternate route, using OLS
Note: not clear why the 5-variable model that was chosen has a higher SSR than the model that only has `HRPA`

In [ ]:
# Import regression modules
# ols - stands for Ordinary least squares, we'll use this
import statsmodels.api as sm
from statsmodels.formula.api import ols
stats2002 = stats[stats['yearID'] == 2002]
m = ols('W ~ HRPA + SinglesPA + DoublesPA + TriplesPA + BBPA ', data = stats2002, ).fit()

# compute the average squared residuals from predictions to years past 2002
stats2003on = stats[stats['yearID'] > 2002]
stats2003on['OPW'] = m.predict( stats2003on )
#stats2003on['OPW2'] = m.predict( stats2003on.drop('W', axis='columns') )
# SinglesPA : 47492.0
# DoublesPA : 38433.0 
# TriplesPA : 38785.0
# HRPA : 33889.0 *
# SinglesPA + DoublesPA : 43230.0
# SinglesPA + TriplesPA : 50232.0
# SinglesPA + HRPA : 42574.0
# DoublesPA + TriplesPA: 38441.0
# DoublesPA + HRPA : 34774.0
# TriplesPA + HRPA : 33885.0 **
# SinglesPA + DoublesPA + TriplesPA : 46297.0
# SinglesPA + DoublesPA + HRPA : 39171.0 
# SinglesPA + TriplesPA + HRPA : 43384.0
# DoublesPA + TriplesPA + HRPA : 34717.0
# SinglesPA + DoublesPA + TriplesPA + HRPA : 40424.0
# HRPA + SinglesPA + DoublesPA + TriplesPA + BBPA : 39495.0

# SinglesPA + DoublesPA + TriplesPA + HRPA + BBPA

print round(pow(stats2003on['W']-stats2003on['OPW'], 2).sum())
print round(((stats2003on['W']-stats2003on['OPW']) ** 2).sum())
print round(((clf.predict(testX)-stats2003on['OPW']) ** 2).sum())
print round(((stats2003on['W']-testY.values) ** 2).sum())
print m.summary()

In [ ]:
sns.regplot(y="OPW", x="W", data=stats2003on, fit_reg = True, color = 'blue')
sns.regplot(x="W", y="W", data=stats2003on, fit_reg= True, color = 'gray', scatter=False)

In [ ]:
plt.scatter(stats2002['W'], m.fittedvalues)
plt.xlabel("Wins: $Y_i$")
plt.ylabel("Predicted wins: $\hat{Y}_i$")
plt.title("Wins vs Predicted Wins: $Y_i$ vs $\hat{Y}_i$")
plt.plot( [70,95], [70,95], c='gray', ls='--' ) # reference line, manual 
plt.show()

In [ ]:
m.ssr
#m.summary()

** Your answer here: **

#### Problem 1(g)

Now we will create a similar database for individual players. Consider only player/year combinations in which the player had at least 500 plate appearances. Consider only the years we considered for the calculations above (after 1947 and seasons with 162 games). For each player/year compute singles, doubles, triples, HR, BB per plate appearance rates. Create a new pandas DataFrame called `playerstats` that has the playerID, yearID and the rates of these stats.  Remove the average for each year as for these rates as done in Problem 1(e). 

In [ ]:
print(players.shape)
players.head()

In [ ]:
# filter pre-1947 and != 162 games
fplayers = players[(players.yearID > 1947) & (players.AB + players.BB > 500)] # shape = (984, 48)

# Compute the per plate appearance rates for singles, doubles, triples, HR, and BB. 
# Create a new pandas DataFrame called stats that has the teamID, yearID, wins and these rates.
# Hint: Singles are hits that are not doubles, triples, nor HR. Plate appearances are base on balls plus at bats.

fplayers = fplayers[['playerID','yearID', 'H', '2B', '3B', 'HR', 'AB', 'BB']]

# PA = BB + AB
# Singles = (H - (2B + 3B + HR))

fplayers['PA'] = fplayers.AB + fplayers.BB
fplayers['Singles'] = fplayers.H - (fplayers['2B'] + fplayers['3B'] + fplayers.HR)

for col in ['Singles', '2B', '3B', 'HR', 'BB']:
    fplayers[col] = fplayers[col] / fplayers['PA']

fplayers.rename(columns={'Singles':'SinglesPA', 
                              '2B':'DoublesPA', 
                              '3B':'TriplesPA', 
                              'HR':'HRPA', 
                              'BB':'BBPA'}, inplace = True)
playerstats = fplayers[['playerID','yearID', 'SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']]

playerstats.head()

In [ ]:
for col in ['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']:
    playerstats.plot(x='yearID', y=col, kind='scatter', alpha=0.1, title=col)

In [ ]:
print(playerstats[['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']].mean().apply(lambda x: round(x, 3)))
playerstats[['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']] = playerstats.groupby('yearID')[['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']].apply(lambda x: x-x.mean())
print(playerstats[['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']].mean().apply(lambda x: round(x, 3)))

# wouldn't "Centralize" be a better name?
#def meanNormalizeRates(df):
#        subRates = df[["1B","2B","3B","HR","BB"]]
#        df[["1B","2B","3B","HR","BB"]] = subRates - subRates.mean(axis=0)
#        return df
#stats = stats.groupby('yearID').apply(meanNormalizeRates)

In [ ]:
for col in ['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']:
    playerstats.plot(x='yearID', y=col, kind='scatter', alpha=0.1, title=(col+', Centered'), color='orange')

Show the head of the `playerstats` DataFrame. 

In [ ]:
playerstats.head()

#### Problem 1(h)

Using the `playerstats` DataFrame created in Problem 1(g), create a new DataFrame called `playerLS` containing the player's lifetime stats. This DataFrame should contain the playerID, the year the player's career started, the year the player's career ended and the player's lifetime average for each of the quantities (singles, doubles, triples, HR, BB). For simplicity we will simply compute the average of the rates by year (a more correct way is to go back to the totals). 

Note (dai): use debut date and final game date from master df

In [ ]:
#playerstats.groupby('playerID', as_index=False)['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA'].mean()
playerLS = playerstats.groupby('playerID', as_index=False).agg({
        'yearID': [np.min, np.max],
        'SinglesPA': np.mean,
        'DoublesPA': np.mean,
        'TriplesPA': np.mean, 
        'HRPA': np.mean, 
        'BBPA': np.mean
    })

Show the head of the `playerLS` DataFrame. 

In [ ]:
playerLS.head()

#### Problem 1(i)

Compute the OPW for each player based on the average rates in the `playerLS` DataFrame. You can interpret this summary statistic as the predicted wins for a team with 9 batters exactly like the player in question. Add this column to the playerLS DataFrame. Call this colum OPW.

In [ ]:
#plt.scatter(trainY, clf.predict(trainX))
playerLS['OPW'] = clf.predict(playerLS[['SinglesPA', 'DoublesPA', 'TriplesPA', 'HRPA', 'BBPA']])
playerLS.head()

#### Problem 1(j)

Add four columns to the `playerLS` DataFrame that contains the player's position (C, 1B, 2B, 3B, SS, LF, CF, RF, or OF), first name, last name and median salary. 

Note (dai): Pick player's position based on maximum count of plays in that position.

In [ ]:
print(playerLS.shape)
playerLS = pd.merge(playerLS, 
         medianSalaries,
         how='left', on='playerID', )
print(playerLS.shape)

In [ ]:
pd.merge(playerLS, 
         fielding[['playerID', 'POS']],
         how='left', on='playerID', )

In [ ]:
playerLS = pd.merge(playerLS, 
         fielding[['playerID', 'POS']],
         how='left', on='playerID', )
print(playerLS.shape)

In [ ]:
playerLS.head(20)

In [ ]:
fielding.info()

In [ ]:
fielding.head()

In [ ]:
master.info()

Show the head of the `playerLS` DataFrame. 

In [ ]:
### Your code here ###

#### Problem 1(k)

Subset the `playerLS` DataFrame for players active in 2002 and 2003 and played at least three years. Plot and describe the relationship bewteen the median salary (in millions) and the predicted number of wins. 

In [ ]:
### Your code here ###

#### Problem 1(l)
Pick one players from one of each of these 10 position C, 1B, 2B, 3B, SS, LF, CF, RF, DH, or OF keeping the total median salary of all 10 players below 20 million. Report their averaged predicted wins and total salary.

In [ ]:
### Your code here ###

#### Problem 1(m)
What do these players outperform in? Singles, doubles, triples HR or BB?

In [ ]:
### Your code here ###

** Your answer here: **

## Discussion for Problem 1

*Write a brief discussion of your conclusions to the questions and tasks above in 100 words or less.*

---

# Problem 2:  $k$-Nearest Neighbors and Cross Validation 

What is the optimal $k$ for predicting species using $k$-nearest neighbor classification 
on the four features provided by the iris dataset.

In this problem you will get to know the famous iris data set, and use cross validation to select the optimal $k$ for a $k$-nearest neighbor classification. This problem set makes heavy use of the [sklearn](http://scikit-learn.org/stable/) library. In addition to Pandas, it is one of the most useful libraries for data scientists! After completing this homework assignment you will know all the basics to get started with your own machine learning projects in sklearn. 

Future lectures will give further background information on different classifiers and their specific strengths and weaknesses, but when you have the basics for sklearn down, changing the classifier will boil down to exchanging one to two lines of code.

The data set is so popular, that sklearn provides an extra function to load it:

In [ ]:
#load the iris data set
iris = sklearn.datasets.load_iris()

X = iris.data  
Y = iris.target

print X.shape, Y.shape

#### Problem 2(a) 
Split the data into a train and a test set. Use a random selection of 33% of the samples as test data. Sklearn provides the [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html) function for this purpose. Print the dimensions of all the train and test data sets you have created. 

In [ ]:
### Your code here ###

#### Problem 2(b)

Examine the data further by looking at the projections to the first two principal components of the data. Use the [`TruncatedSVD`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) function for this purpose, and create a scatter plot. Use the colors on the scatter plot to represent the different classes in the target data. 

In [ ]:
### Your code here ###

#### Problem 2(c) 

In the lecture we discussed how to use cross validation to estimate the optimal value for $k$ (the number of nearest neighbors to base the classification on). Use ***ten fold cross validation*** to estimate the optimal value for $k$ for the iris data set. 

**Note**: For your convenience sklearn does not only include the [KNN classifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), but also a [grid search function](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV). The function is called grid search, because if you have to optimize more than one parameter, it is common practice to define a range of possible values for each parameter. An exhaustive search then runs over the complete grid defined by all the possible parameter combinations. This can get very computation heavy, but luckily our KNN classifier only requires tuning of a single parameter for this problem set. 

In [ ]:
### Your code here ###

#### Problem 2(d)

Visualize the result by plotting the score results versus values for $k$. 

In [ ]:
### Your code here ###

Verify that the grid search has indeed chosen the right parameter value for $k$.

In [ ]:
### Your code here ###

#### Problem 2(e)

Test the performance of our tuned KNN classifier on the test set.

In [ ]:
### Your code here ###

## Discussion for Problem 2

*Write a brief discussion of your conclusions to the questions and tasks above in 100 words or less.*

---

# Problem 3: The Curse and Blessing of Higher Dimensions

In this problem we will investigate the influence of higher dimensional spaces on the classification. The data set is again one of the standard data sets from sklearn. The [digits data set](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html) is similar to the MNIST data set discussed in the lecture. The main difference is, that each digit is represented by an 8x8 pixel image patch, which is considerably smaller than the 28x28 pixels from MNIST. In addition, the gray values are restricted to 16 different values (4 bit), instead of 256 (8 bit) for MNIST. 

First we again load our data set.

In [ ]:
digits = sklearn.datasets.load_digits()

X = digits.data  
Y = digits.target

print X.shape, Y.shape

#### Problem 3(a) 

Start with the same steps as in Problem 2. Split the data into train and test set. Use 33% of the samples as test data. Print the dimensions of all the train and test data sets you created. 

In [ ]:
### Your code here ###

#### Problem 3(b) 

Similar to Problem 2(b), create a scatter plot of the projections to the first two PCs.  Use the colors on the scatter plot to represent the different classes in the target data. How well can we separate the classes?

**Hint**: Use a `Colormap` in matplotlib to represent the diferent classes in the target data. 

In [ ]:
### Your code here ###

Create individual scatter plots using only two classes at a time to explore which classes are most difficult to distinguish in terms of class separability.  You do not need to create scatter plots for all pairwise comparisons, but at least show one. 

In [ ]:
### Your code here ###

Give a brief interpretation of the scatter plot. Which classes look like hard to distinguish? Do both feature dimensions contribute to the class separability? 

** Your answer here: **

#### Problem 3(c) 

Write a **ten-fold cross validation** to estimate the optimal value for $k$ for the digits data set. *However*, this time we are interested in the influence of the number of dimensions we project the data down as well. 

Extend the cross validation as done for the iris data set, to optimize $k$ for different dimensional projections of the data. Create a boxplot showing test scores for the optimal $k$ for each $d$-dimensional subspace with $d$ ranging from one to ten. The plot should have the scores on the y-axis and the different dimensions $d$ on the x-axis. You can use your favorite plot function for the boxplots. [Seaborn](http://web.stanford.edu/~mwaskom/software/seaborn/index.html) is worth having a look at though. It is a great library for statistical visualization and of course also comes with a [`boxplot`](http://web.stanford.edu/~mwaskom/software/seaborn/generated/seaborn.boxplot.html) function that has simple means for changing the labels on the x-axis.

In [ ]:
### Your cross validation and evaluation code here ###

In [ ]:
### Your boxplot code here ### 

Write a short interpretation of the generated plot, answering the following questions:

* What trend do you see in the plot for increasing dimensions?

* Why do you think this is happening?

** Your answer here: **

#### Problem 3(d) 

**For AC209 Students**: Change the boxplot we generated above to also show the optimal value for $k$ chosen by the cross validation grid search. 

In [ ]:
### Your code here ### 

Write a short interpretation answering the following questions:

* Which trend do you observe for the optimal value of $k$?

* Why do you think this is happening?

** Your answer here: **

## Discussion for Problem 3

*Write a brief discussion of your conclusions to the questions and tasks above in 100 words or less.*

---

# Submission Instructions

To submit your homework, create a folder named **lastname_firstinitial_hw#** and place your IPython notebooks, data files, and any other files in this folder. Your IPython Notebooks should be completely executed with the results visible in the notebook. We should not have to run any code.  Compress the folder (please use .zip compression) and submit to the CS109 dropbox in the appropriate folder. *If we cannot access your work because these directions are not followed correctly, we will not grade your work.*
